<a name="top"></a>
<div style="width:1000 px">

<div style="float:right; width:98 px; height:98px;">
<img src="https://cdn.miami.edu/_assets-common/images/system/um-logo-gray-bg.png" alt="Miami Logo" style="height: 98px;">
</div>

<div style="float:right; width:98 px; height:98px;">
<img src="https://media.licdn.com/dms/image/C4E0BAQFlOZSAJABP4w/company-logo_200_200/0/1548285168598?e=2147483647&v=beta&t=g4jl8rEhB7HLJuNZhU6OkJWHW4cul_y9Kj_aoD7p0_Y" alt="STI Logo" style="height: 98px;">
</div>


<h1>Calculate Surface-Based Hot Dry Windy for the UFS S2S Model</h1>
By: Kayla Besong, PhD
    <br>
Last Edited: 11/29/23
<br>
<br>    
<br>
Takes models/variables downloaded and calculates surface based hot-dry-windy. The hot-dry-windy calculation uses vapor pressure deficit and multiplies by windspeed, hence leveraging previously calculated variables. The function that computes the 24HR AVG, MIN, MAX outputs is in File_concat_mod_functions.ipynb. 
<br>
<br>
NOTE: The operational and 'true' hot-dry-windy index (HDWI) is not computed at the surface, rather it involves analyzing vpd and windspeed in the lowest 500m of the atmosphere and is  more computationally intensive. The 'true' HDWI also takes the max value of the day. Here, by just multiplying vpd by windspeed at the surface, the resulting product is a 'surface-based-hot-dry-windy'. The difference between surface based HDW and the HDWI can be stark depending on the region you are analyzing. Please see: (Kramer et al., 2024; Watts et al., 2020).
<br>
<div style="clear:both"></div>
</div>

<hr style="height:2px;">

## Import needed libraries, etc.

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
from dask.distributed import Client, LocalCluster
import dask.array as da
import os
import glob
from metpy.units import units
import math

In [3]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
pd.options.mode.chained_assignment = None

## OPTIONAL: Establish a dask client. This is a lot of data.

In [4]:
Cluster = LocalCluster(n_workers = 8, threads_per_worker=4, memory_limit='30GB',  processes=True)
#Cluster = LocalCluster()

In [5]:
client = Client(Cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:34197/status,
Dashboard: http://127.0.0.1:34197/status,Workers: 8
Total threads: 32,Total memory: 223.52 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:41580,Workers: 8
Dashboard: http://127.0.0.1:34197/status,Total threads: 32
Started: Just now,Total memory: 223.52 GiB
Comm: tcp://127.0.0.1:43152,Total threads: 4
Dashboard: http://127.0.0.1:40164/status,Memory: 27.94 GiB
Nanny: tcp://127.0.0.1:43998,


### The integral notebook of functions to run

In [6]:
%run ../../../Universal_Functions/File_concat_mod_functions.ipynb

# Calculate HDW 

In [7]:
model = 'UFS_S2S'
#p = 8
ps = [6, 7, 8]        # define which prototype 
ps = [5]        # define which prototype 

In [8]:
var_list = ['wspeed', 'vpd']
var_list

['wspeed', 'vpd']

In [9]:
main_dir = '../../../../database_files'

In [10]:
def resampler_UFS_hdw(v, var_file_cc, output_dir):
     
    model_save_str = get_filename('UFS_S2S')

    try:

        print(f'starting {i} {v}')
               
        model_abs = var_file_cc[v].resample(valid_time='24H').mean(dim='valid_time', skipna = True).to_dataset()
        model_min = var_file_cc[v].resample(valid_time='24H').min(dim='valid_time', skipna = True).to_dataset()
        model_max = var_file_cc[v].resample(valid_time='24H').max(dim='valid_time', skipna = True).to_dataset()
                    
        # save_file(model_abs, f'{i}', f'{v}_UFS_S2S_FORECAST_AVG_Daily_{i.split('/')[-1]}.nc')
        # save_file(model_min, f'{i}', f'{v}_UFS_S2S_FORECAST_MIN_Daily_{i.split('/')[-1]}.nc')
        # save_file(model_max, f'{i}', f'{v}_UFS_S2S_FORECAST_MAX_Daily_{i.split('/')[-1]}.nc')
        # save_file(var_file_cc, f'{i}', f'{v}_UFS_S2S_FORECAST_Abs_{i.split('/')[-1]}.nc')

        save_file(model_abs, f'{i}', f'{v}AVG_{model_save_str}_Daily_{i.split('/')[-1]}.nc')
        save_file(model_min, f'{i}', f'{v}MIN_{model_save_str}_Daily_{i.split('/')[-1]}.nc')
        save_file(model_max, f'{i}', f'{v}MAX_{model_save_str}_Daily_{i.split('/')[-1]}.nc')
        save_file(var_file_cc, f'{i}', f'{v}_{model_save_str}_Abs_{i.split('/')[-1]}.nc')

    except OSError:
        print(f'{v} not available for {i}')

In [11]:
%%time

for p in ps:

    parent_dirs = glob.glob(os.path.join(f'{main_dir}/{model}/{p}/', '*'))

    for i in parent_dirs:
                    
        w_files = glob.glob(os.path.join(f'{i}/', f'*{var_list[0]}*Abs*'))[0]                      # read in vpd, windspeed 
        vpd_files = glob.glob(os.path.join(f'{i}/', f'*{var_list[1]}*Abs*'))[0]
    
        w_file_cc = xr.open_dataset(w_files).chunk(get_chunk(model))
        vpd_file_cc = xr.open_dataset(vpd_files).chunk(get_chunk(model))
        
        w_file_cc = w_file_cc.sortby('valid_time')
        vpd_file_cc = vpd_file_cc.sortby('valid_time')
    
        hdwi = (vpd_file_cc['vpd']*w_file_cc['wspeed']).to_dataset(name = 'hdw')                   # calculate the surface based hot-dry-windy
    
        resampler_UFS_hdw('hdw', hdwi.chunk(get_chunk(model)), main_dir)                           # call the special UFS min, max, avg processor in File_concat_mod_functions.ipynb


starting ../../../../database_files/UFS_S2S/5/20170101 hdw
starting ../../../../database_files/UFS_S2S/5/20170115 hdw
starting ../../../../database_files/UFS_S2S/5/20170201 hdw
starting ../../../../database_files/UFS_S2S/5/20170215 hdw
starting ../../../../database_files/UFS_S2S/5/20170301 hdw
starting ../../../../database_files/UFS_S2S/5/20170315 hdw
starting ../../../../database_files/UFS_S2S/5/20170401 hdw
starting ../../../../database_files/UFS_S2S/5/20170415 hdw
starting ../../../../database_files/UFS_S2S/5/20170501 hdw
starting ../../../../database_files/UFS_S2S/5/20170515 hdw
starting ../../../../database_files/UFS_S2S/5/20170601 hdw
starting ../../../../database_files/UFS_S2S/5/20170615 hdw
starting ../../../../database_files/UFS_S2S/5/20170701 hdw
starting ../../../../database_files/UFS_S2S/5/20170715 hdw
starting ../../../../database_files/UFS_S2S/5/20170801 hdw
starting ../../../../database_files/UFS_S2S/5/20170815 hdw
starting ../../../../database_files/UFS_S2S/5/20170901 h